In [13]:
import pandas as pd
import random
from rdflib.namespace import RDF, RDFS, FOAF, XSD
from rdflib import Graph
import pandas as pd
from rdflib import Namespace
from rdflib import Literal

In [24]:
authorsDF = pd.read_csv('data/authors.csv')
authorsPapersDF = pd.read_csv('data/authorsPapers.csv')
chairsDF = pd.read_csv('data/chairs.csv')
conferencesDF = pd.read_csv('data/conferences.csv')
editorsDF = pd.read_csv('data/editors.csv')
subjectDomainDF = pd.read_csv('data/subjectDomain.csv')
finalPapersDF = pd.read_csv('data/finalPapers.csv')
journalsDF = pd.read_csv('data/journals.csv')
papersDF = pd.read_csv('data/papers.csv')
reviewersDF = pd.read_csv('data/reviewers.csv')
reviewsDF = pd.read_csv('data/reviews.csv')
subjectsPapersDF = pd.read_csv('data/subjectsPapers.csv')
submittedPapersDF = pd.read_csv('data/submittedPapers.csv')

In [16]:
authorsPapersDF.head(2)

,aUrlId,pId,aId,aName,paperUrlId,conferenceJournalId,conferenceJournalTitle,pTitle,abstract,pType
0,6391996,p0,a0,E. Bolyen,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,shortPaper
1,6307534,p0,a1,J. Rideout,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,shortPaper


In [9]:
# Creating a graph
graph = Graph()

In [10]:
# Create many URIRefs in the same namespace, i.e. URIs with the same prefix
LAB = Namespace("http://SDM_LAB3.org/")

In [11]:
# Bind the lab namespace to a prefix for more readable output
graph.bind('lab',LAB)

#### Author -- [writes] --> Paper

In [33]:
authorsPapersDF.head(2)

,aUrlId,pId,aId,aName,paperUrlId,conferenceJournalId,conferenceJournalTitle,pTitle,abstract,pType
0,6391996,p0,a0,E. Bolyen,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,shortPaper
1,6307534,p0,a1,J. Rideout,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,shortPaper


In [17]:
# Adding author--[writes]--> paper instances to the graph
g.add((LAB.Author, LAB.writes, LAB.Paper))   
for k in range(len(authorsPapersDF['aName'])):
    g.add((Literal(authorsPapersDF['aName'][k]), LAB.writes, Literal(authorsPapersDF['pTitle'][k])))

#### Paper -- [submitted] --> SubmittedPaper

In [32]:
submittedPapersDF.head(2)

,sId,pId,pTitle,abstract,conferenceJournalId,conferenceJournalTitle,decision
0,s0,p0,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,j0,Nature Biotechnology,accepted
1,s1,p1,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ...",c90,Computer Vision and Pattern Recognition,accepted


In [41]:
# Adding paper--[submitted]--> submittedPaper instances to the graph
g.add((LAB.Paper, LAB.submitted, LAB.SubmittedPaper))   
for k in range(len(submittedPapersDF['pTitle'])):
    g.add((Literal(submittedPapersDF['pTitle'][k]), LAB.writes, Literal(submittedPapersDF['pTitle'][k])))

#### Paper -- [paperRelatedTo] --> SubjectDomain

In [42]:
subjectsPapersDF.head(2)

,subjectDomainName,pId,dId,paperUrlId,conferenceJournalId,conferenceJournalTitle,pTitle,abstract,pType
0,Engineering,p0,d0,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,shortPaper
1,Medicine,p0,d1,0c2d3b28d48426b8b72f7214a7708ba8b4efa9d6,j0,Nature Biotechnology,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,shortPaper


In [30]:
# Adding paper--[paperRelatedTo]--> subjectDomain instances to the graph
g.add((LAB.Paper, LAB.paperRelatedTo, LAB.SubjectArea))
for k in range(len(subjectsPapersDF['pTitle'])):
    g.add((Literal(subjectsPapersDF['pTitle'][k]), LAB.paperRelatedTo, Literal(subjectsPapersDF['subjectDomainName'][k])))

#### Conference -- [isIn] --> Proceedings

In [35]:
conferencesDF.head(2)

,cId,cUrlId,cTitle,cType,cProceedings
0,c0,b55b50b1-aae7-47a7-b042-8aecc930073d,International Conference on Human Factors in C...,symposium,10
1,c1,2c9ecac6-f875-4a9b-acc2-10bd9f6782df,Technical Symposium on Computer Science Education,workshop,1


In [36]:
# Adding conference--[isIn]--> proceedings instances to the graph
g.add((LAB.Conference, LAB.isIn, LAB.Proceedings))
for k in range(len(conferencesDF['cTitle'])):
    g.add((Literal(conferencesDF['cTitle'][k]), LAB.isIn, Literal(conferencesDF['cProceedings'][k])))

#### Journal -- [isOf] --> Volume

In [37]:
journalsDF.head(2)

,jId,jUrlId,jTitle,jVolumes
0,j0,458166b3-de17-4bf3-bbbb-e53782de2f0f,Nature Biotechnology,4
1,j1,c6840156-ee10-4d78-8832-7f8909811576,IEEE Transactions on Knowledge and Data Engine...,0


In [39]:
# Adding journal--[isOf]--> volume instances to the graph
g.add((LAB.Journal, LAB.isOf, LAB.Volume))
for k in range(len(journalsDF['jTitle'])):
    g.add((Literal(journalsDF['jTitle'][k]), LAB.isOf, Literal(journalsDF['jVolumes'][k])))

#### SubmittedPaper -- [isSubmittedTo] --> Venue

In [40]:
submittedPapersDF.head(2)

,sId,pId,pTitle,abstract,conferenceJournalId,conferenceJournalTitle,decision
0,s0,p0,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,j0,Nature Biotechnology,accepted
1,s1,p1,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ...",c90,Computer Vision and Pattern Recognition,accepted


In [43]:
# Adding submittedPaper--[isSubmittedTo]--> venue instances to the graph
g.add((LAB.SubmittedPaper, LAB.isSubmittedTo, LAB.Venue))
for k in range(len(submittedPapersDF['pTitle'])):
    g.add((Literal(submittedPapersDF['pTitle'][k]), LAB.isSubmittedTo, Literal(submittedPapersDF['conferenceJournalTitle'][k])))

#### SubmittedPaper -- [isAs] --> FinalPaper

In [45]:
finalPapersDF.head(2)

,sId,pId,pTitle,abstract,conferenceJournalId,conferenceJournalTitle,decision,fId
0,s0,p0,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,j0,Nature Biotechnology,accepted,f0
1,s1,p1,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ...",c90,Computer Vision and Pattern Recognition,accepted,f1


In [46]:
# Adding submittedPaper--[isAs]--> finalPaper instances to the graph
g.add((LAB.SubmittedPaper, LAB.isAs, LAB.FinalPaper))
for k in range(len(finalPapersDF['pTitle'])):
    g.add((Literal(finalPapersDF['pTitle'][k]), LAB.isAs, Literal(finalPapersDF['pTitle'][k])))

#### FinalPaper -- [isPublishedInConference] --> Proceedings & FinalPaper -- [isPublishedInJournal] --> Volume

In [47]:
finalPapersDF.head(2)

,sId,pId,pTitle,abstract,conferenceJournalId,conferenceJournalTitle,decision,fId
0,s0,p0,"Reproducible, interactive, scalable and extens...",Abstract content goes here ...,j0,Nature Biotechnology,accepted,f0
1,s1,p1,High-Dimensional Probability: An Introduction ...,"© 2018, Cambridge University Press Let us summ...",c90,Computer Vision and Pattern Recognition,accepted,f1


In [48]:
conferencesDF.head(2)

,cId,cUrlId,cTitle,cType,cProceedings
0,c0,b55b50b1-aae7-47a7-b042-8aecc930073d,International Conference on Human Factors in C...,symposium,10
1,c1,2c9ecac6-f875-4a9b-acc2-10bd9f6782df,Technical Symposium on Computer Science Education,workshop,1


In [ ]:
# Adding FinalPaper -- [isPublishedInConference] --> Proceedings & FinalPaper -- [isPublishedInJournal] --> Volume instances to the graph
g.add((LAB.FinalPaper, LAB.isPublishedInConference, LAB.Proceedings))
g.add((LAB.FinalPaper, LAB.isPublishedInJournal, LAB.Volume))
for k in range(len(finalPapersDF['pTitle'])):
    if finalPapersDF['conferenceJournalId'][k][0] == 'c':
        g.add((Literal(finalPapersDF['pTitle'][k]), LAB.isPublishedInConference, Literal(Proceedings['pTitle'][k])))
    else:
        g.add((Literal(finalPapersDF['pTitle'][k]), LAB.isPublishedInJournal, Literal(finalPapersDF['pTitle'][k])))

In [ ]:



g.add((KG.Journal, KG.JournalRelatedTo, KG.SubjectArea))
for k in range(len(Journal_Keyword['keyword'])):
    g.add((Literal(Journal_Keyword['journal'][k]), KG.JournalRelatedTo, Literal(Journal_Keyword['keyword'][k])))

g.add((KG.Conference, KG.ConferenceRelatedTo, KG.SubjectArea))
for k in range(len(Paper_volume['keyword'])):
    g.add((Literal(Paper_volume['conference'][k]), KG.ConferenceRelatedTo, Literal(Paper_volume['keyword'][k])))


g.add((KG.Reviewer, KG.decided, KG.Decision))
for k in range(len(Review_Papers['name'])):
    g.add((Literal(Review_Papers['name'][k]), KG.decided, Literal(Review_Papers['decision'][k])))


g.add((KG.Conference, KG.handledBy, KG.Chair))
g.add((KG.Journal, KG.handledBy, KG.Editor))
for k in range(len(Editor_Chair['conference'])):
    if Editor_Chair['confjournal'][k] == 'Conference':
        g.add((Literal(Editor_Chair['conference'][k]), KG.handledBy, Literal(Editor_Chair['name'][k])))
    else:
        g.add((Literal(Editor_Chair['journal'][k]), KG.handledBy, Literal(Editor_Chair['name'][k])))
        
#Not resonable       
g.add((KG.Chair, KG.assigned, KG.Reviewer))
g.add((KG.Editor, KG.assigned, KG.Reviewer))
for k in range(len(Editor_Chair['conference'])):
    if Editor_Chair['confjournal'][k] == 'Conference':
        g.add((Literal(Editor_Chair['name'][k]), KG.assigned, Literal(Editor_Chair['name'][k])))
    else:
        g.add((Literal(Editor_Chair['name'][k]), KG.assigned, Literal(Editor_Chair['name'][k])))
        
#g.add((KG.Reviewer, KG.reviewed, KG.Submission))
#g.add((Literal(Editor_Chair['name'][k]), KG.reviewed, Literal(int(Editor_Chair['date_y'][k]))))

g.serialize(destination=f'/home/vladka/Downloads/output/ABOX.ttl')